# Train a Neural Network loading data from and to SmartSim

In this tutorial, we will see how one can set up a workflow in which one or multiple processes produce data (e.g. in a simulation) and one or multiple processes consume the data to train a neural network. The key to achieve this behavior is to use SmartSim to load data from and to the database.

This tutorial works on Slurm, but can easily be adapted to any supported Workload Manager (PBS, Cobalt, or LSF), with an only difference: in Slurm, we are allocating the resources *from* this Notebook, whereas with other WLMs, this Notebook has to be started *within* an existing allocation.

Note: this tutorial requires the python packages `torch`, `mpi4py` and `horovod`.

## 1. First scenario: an ensemble of parallel producers and a single trainer

The first use case is similar to a common workflow:
- several copies of a simulation (possibly with different initializations) are running, each one consisting of multiple MPI ranks. Each rank produces samples (e.g. data points computed by the simulation) at regular intervals (e.g. each time iteration)
- a neural network has to be trained on the data produced by the simulation, and as new data is produced, the neural network needs to add it to its training data set.


### 1.1 Workflow components
We will use SmartSim to allow the exchange of data between the data-producing processes and the training service. Thus, the first component which we will need to launch is the `Orchestrator`. This is a fairly small example, thus we will run a single-node DB.

In [ ]:
from smartsim import Experiment
from smartsim.database import SlurmOrchestrator
from smartsim.settings import SrunSettings
from smartsim import slurm

def launch_cluster_orc(experiment, port, alloc):
    """Just spin up a database cluster"""

    db = SlurmOrchestrator(port=port,
                            db_nodes=1,
                            batch=False,
                            alloc=alloc,
                            interface="ib0")
    

    # generate directories for output files
    # pass in objects to make dirs for
    experiment.generate(db, overwrite=True)

    # start the database on interactive allocation
    experiment.start(db, block=True)

    return db


Our data will be very simple: each rank will produce random samples, drawing values from a Gaussian distribution centered at the rank id, and each sample will be labeled with the rank id. The neural network will be trained to infer which rank produced a data sample. Thus, our data set will have a total number of labels equal to the number of ranks in each replicas.

The data-producing processes will be started as part of an ensemble of two replicas (this mimics the execution of two copies of the simulation). The actual script producing the data is called `data_uploader.py` and is contained in the `torch` directory. Its content is

```python

from smartsim.ml import TrainingDataUploader
from os import environ
from time import sleep
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
mpi_rank = comm.Get_rank()
mpi_size = comm.Get_size()

batches_per_loop = 10

data_uploader = TrainingDataUploader(num_classes=mpi_size,
                                     cluster=False, 
                                     producer_prefixes="uploader",
                                     num_ranks=mpi_size)
                                     
if environ["SSKEYOUT"] == "uploader_0":
    data_uploader.publish_info()


# Start "simulation", produce data every two minutes, for thirty minutes
for _ in range(15):
    new_batch = np.random.normal(loc=float(mpi_rank), scale=5.0, size=(32*batches_per_loop, 224, 224, 3)).astype(float)
    new_labels = np.ones(shape=(32*batches_per_loop,)).astype(int) * mpi_rank

    data_uploader.put_batch(new_batch, new_labels)
    print(f"{mpi_rank}: New data pushed to DB")
    sleep(120)

```

as the script is running in python, we can use the `TrainingDataUploader` class, which streamlines uploading of the data samples from the simulation. Here is an explanation of the arguments:
- `num_classes` is the number of classes of our training data set
- `smartredis_cluster=False` specifies that the DB is not a cluster (it is a single shard)
- `producer_prefixes` is used to identify the SmartSim entity names of the processes producing data: this is useful in the case the training service has several incoming entities (as defined in SmartSim, through the environment variable `SSKEYIN`), but it should expect data only from a subset of them. 
- `num_ranks` is the number of concurrent data loaders withing this application, in this case it is simply the number of MPI ranks, as each process will upload its own data.


At each iteration, each rank of "simulation" calls `put_batch` to upload a batch of samples and the corresponding labels. The batch will be uploaded on the DB and stored under a key composed by a prefix (defaulting to `samples`), the sub-index, and the iteration number, which increases monotonically every time a batch is uploaded.

Now that we know the content of `data_uploader.py`, we can create an entity representing the uploader ensemble.

In [ ]:
def create_uploader(experiment, alloc, nodes=1, tasks_per_node=1):
   """Start an ensemble of two processes producing sample batches at
      regular intervals.
   """
   srun = SrunSettings(exe="python",
                     exe_args="data_uploader.py",
                     env_vars={"PYTHONUNBUFFERED": "1"},
                     alloc=alloc)
   srun.set_nodes(nodes)
   srun.set_tasks_per_node(tasks_per_node)

   uploader = experiment.create_ensemble("uploader", replicas=2, run_settings=srun)

   # create directories for the output files and copy
   # scripts to execution location inside newly created dir
   # only necessary if its not an executable (python is executable here)
   uploader.attach_generator_files(to_copy=["./torch/data_uploader.py"])
   experiment.generate(uploader, overwrite=True)
   return uploader
   

The last component of our workflow is the trainer. This process will keep downloading samples as they are produced, and use them to train a neural network.

Here is the content of the `training_service.py` script, stored in the `torch` directory

```python

import numpy as np
import torchvision.models as models
from smartsim.ml.torch import DynamicDataGenerator, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

if __name__ == '__main__':
    torch.multiprocessing.set_start_method('spawn')
    training_set = DynamicDataGenerator(cluster=False,
                                 init_samples=False,
                                 verbose=True)
    trainloader = DataLoader(training_set, batch_size=None,
                             num_workers=2)
    model = models.mobilenet_v2().double().to('cuda')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    print("Started training")

    for epoch in range(50):  # loop over the dataset multiple times

        running_loss = 0.0
        epoch_running_loss = 0.0
        output_period = 100
        print(f"Epoch {epoch}")
        for i, data in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].double().to('cuda'), data[1].to('cuda')
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_running_loss += loss.item()

            if i % output_period == (output_period-1):    # print every "output_period" mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / output_period))
                running_loss = 0.0

        
        print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, epoch_running_loss / (i+1)))
        epoch_running_loss = 0.0
                
    print('Finished Training')
    
```

The key component is the `DataGenerator` object. Since the DB contains the metadata of the `TrainingDataUploader`, and these are stored under a default key, the `DataGenerator` will retrieve them and use them to know the key of the uploaded batches of samples and labels. We need to specify `init_samples=False`, because initialization of the data set will be performed by the `DataLoader` workers: notice that the `DataLoader` is imported from `smartsim.ml.torch`: it is our custom implementation of a data loader, which will also triger a data update at the beginning of each epoch, to check if there are new samples available.

We create a SmartSim entity representing the trainer and we are ready to go!

In [ ]:
def create_trainer(experiment, alloc):
    """Start a process running a training service which will
       download batches from the DB.
    """
    srun = SrunSettings(exe="python",
                        exe_args="training_service.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_tasks(1)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./torch/training_service.py")
    experiment.generate(trainer, overwrite=True)
    return trainer
    

### 1.2 Request an allocation

We need one node for the DB, two for the producer ensemble, and one for the trainer, thus we request 4 nodes to be allocated.

In [ ]:
alloc = slurm.get_allocation(nodes=4, time="03:00:00", options={"constraint": "V100", "partition": "spider"})

### 1.3 Run the workflow

Now that all components are available, we create the SmartSim experiment representing our workflow. 
Notice that the line
```python
uploader_model.enable_key_prefixing()
```
sets the ``uploader`` process so that tensor keys produced within it will be prefixed with its
SmartSim entity name, and the lines
```python
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)
```
make sure that each uploader replica (within the ensemble) is set as an incoming entity of the `trainer_model`. This will allow the `trainer_model` to know which processes are producing batches it will need to download.

We call `exp.start` and the workflow is launched! As the trainer was started with `verbose=True`, we can look at its output in `launch_streaming`: we will see that it keeps downloading batches at the end of each epoch, as expected. 

In [ ]:
exp = Experiment("launch_streaming", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 2)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer(exp, alloc)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

### 1.4 Stop the workflow (optional) and release the allocation
If we did not wait until completion of the previous cell, but we stopped it, we need to stop the SmartSim entities. 

In [ ]:
exp.stop(db, uploader_model, trainer_model)

Finally, we release the allocation.

In [ ]:
slurm.release_allocation(alloc)

## 2 Second scenario: an ensemble of parallel producers and a distributed trainer

In the second scenario, we use Horovod to distribute the training and use multiple ranks. Each rank will download only a portion of the dataset, thus speeding up the download and training process.


### 2.1 Workflow components
The only component we change with respect to the previous example, is the training service, which is now defined in `training_service_hvd.py`:

```python

import numpy as np
import torchvision.models as models

from smartsim.ml.torch import DynamicDataGenerator, DataLoader

import torch
import torch.nn as nn
import torch.optim as optim

import horovod.torch as hvd

if __name__ == '__main__':

    # Initialize Horovod
    hvd.init()

    hvd_rank = hvd.rank()
    hvd_size = hvd.size()

    # Pin GPU to be used to process local rank (one GPU per process)
    torch.cuda.set_device(hvd.local_rank())

    torch.multiprocessing.set_start_method('spawn')
    training_set = DynamicDataGenerator(cluster=False,
                                 verbose=True,
                                 init_samples=False,
                                 num_replicas=hvd_size,
                                 replica_rank=hvd_rank)

    trainloader = DataLoader(training_set, batch_size=None,
                             num_workers=2)

    model = models.mobilenet_v2().double().to('cuda')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001*hvd_size)
    optimizer = hvd.DistributedOptimizer(optimizer, named_parameters=model.named_parameters())
    hvd.broadcast_parameters(model.state_dict(), root_rank=0)
    print(f"Rank {hvd_rank}: Started training")

    for epoch in range(100):  # loop over the dataset multiple times

        running_loss = 0.0
        epoch_running_loss = 0.0
        if hvd_rank == 0:
            print(f"Epoch {epoch}")
        output_period = 100

        for i, data in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].double().to('cuda'), data[1].to('cuda')
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_running_loss += loss.item()

            if hvd_rank == 0:
                if i % output_period == (output_period-1):    # print every "output_period" mini-batches
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / output_period))
                    running_loss = 0.0

        if hvd_rank == 0:    
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, epoch_running_loss / (i+1)))
            epoch_running_loss = 0.0
                
    print('Finished Training')
    
```

Compared to the previous training service, we notice that the only thing changing for the `DataGenerator` is that we need to specify the total number of replicas, which is equal to the total number of Horovod ranks, and the rank of each replica, which corresponds to the Horovod rank. The rest of the training service script is modified according to the standard Horovod distributed training rules.

Let's define the SmartSim entity representing the training service.

In [ ]:
def create_trainer_hvd(experiment, alloc, nodes=1, tasks_per_node=1):
   """Start a process running a distributed training service which will
      download batches from the DB and use Horovod to distribute
      data and compute global weight updates.
   """
   srun = SrunSettings(exe="python",
                     exe_args="training_service_hvd.py",
                     env_vars={"PYTHONUNBUFFERED": "1"},
                     alloc=alloc)
   srun.set_nodes(nodes)
   srun.set_tasks_per_node(tasks_per_node)

   trainer = experiment.create_model("trainer", srun)

   # create directories for the output files and copy
   # scripts to execution location inside newly created dir
   # only necessary if its not an executable (python is executable here)
   trainer.attach_generator_files(to_copy="./torch/training_service_hvd.py")
   experiment.generate(trainer, overwrite=True)
   return trainer


### 2.2 Request an allocation

As before, we need one node for the DB, two for the producer ensemble, and one for the trainer, thus we request 4 nodes to be allocated.

In [ ]:
alloc = slurm.get_allocation(nodes=4, time="03:00:00", options={"constraint": "V100", "partition": "spider"})

### 2.3 Run the workflow

Now that all components are available, we create the SmartSim experiment representing our workflow. The setup is completely identical to the previous example, thus we can start the experiment and look at the output files to see that now each replica has a smaller portion of the dataset, and the training proceeds much faster!

In [ ]:
exp = Experiment("launch_streaming_hvd", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 16)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer_hvd(exp, alloc, 1, 8)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

### 2.4 Stop the workflow (optional) and release the allocation
If we did not wait until completion of the previous cell, but we stopped it, we need to stop the SmartSim entities. 

In [ ]:
exp.stop(db, uploader_model, trainer_model)

Finally, we release the allocation.

In [ ]:
slurm.release_allocation(alloc)